In [ ]:
from segment_mapping import *
from plpygis import Geometry
import shapely.geometry as sg
import shapely.ops as so
import matplotlib.pyplot as plt
from apperception.utils import fetch_camera_config
import cv2
# importing movie py libraries
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage

In [ ]:
test_config = fetch_camera_config('samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385105912404.jpg', database)

In [ ]:
test_config

In [ ]:
mapping = map_imgsegment_roadsegment(test_config)

In [ ]:
test_file_path = '/home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385105912404.jpg'
frame = cv2.imread(test_file_path)
fig, axs = plt.subplots()
axs.set_aspect('equal', 'datalim')
x_ego, y_ego = test_config['egoTranslation'][:2]
axs.plot(x_ego, y_ego, color='green', marker='o', markersize=5)
colormap = plt.cm.get_cmap('hsv', len(mapping))
i = 0
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
display_video = cv2.VideoWriter('in_videw_display.avi',fourcc, 1, (1600, 900))
for cam_segment, road_segment_info in mapping:
    color = colormap(i)
    xs = [point[0] for point in road_segment_info.segment_polygon]
    ys = [point[1] for point in road_segment_info.segment_polygon]
    segmenttype = road_segment_info.segment_type
    axs.fill(xs, ys, alpha=0.5, fc=color, ec='none')
    axs.text(np.mean(np.array(xs)), np.mean(np.array(ys)), 
             ','.join(segmenttype) if segmenttype and ('lane' in segmenttype or 'intersection' in segmenttype) else '')
    current_plt = mplfig_to_npimage(fig)
    i += 1
    
    fov_lines = road_segment_info.fov_lines
    axs.plot([p[0] for p in fov_lines[0]], [p[1] for p in fov_lines[0]], color='red', marker='o', markersize=2)
    axs.plot([p[0] for p in fov_lines[1]], [p[1] for p in fov_lines[1]], color='red', marker='o', markersize=2)

    
    display_frame = frame.copy()
    cv2.polylines(display_frame, [np.array(cam_segment, np.int32).reshape((-1, 1, 2))], True, (0, 255, 0), 2)
    display_frame[:current_plt.shape[0], :current_plt.shape[1]] = current_plt
    display_video.write(display_frame)
#     cv2.imshow("frame",display_frame)
#     cv2.waitKey(0) 
#     cv2.destroyAllWindows()
display_video.release()
plt.show()

In [ ]:
all_segment_within_distance = construct_search_space(test_config)

In [ ]:
import numpy as np
BLUE = '#6699cc'
GRAY = '#999999'

fig, axs = plt.subplots()
axs.set_aspect('equal', 'datalim')
ego_translation = Geometry(test_config['egoTranslation'].values[0])
x_ego, y_ego = ego_translation.shapely.xy
print(ego_translation.shapely)
axs.plot(x_ego, y_ego, color='green', marker='o', markersize=5)

colormap = plt.cm.get_cmap('hsv', len(all_segment_within_distance))
i = 0
for segmentid, segmentpolygon, segmenttype, _ in all_segment_within_distance:
    color = colormap(i)
    xs, ys = Geometry(segmentpolygon).shapely.exterior.xy
    axs.fill(xs, ys, alpha=0.5, fc=color, ec='none')
    axs.text(np.mean(np.array(xs)), np.mean(np.array(ys)), 
             ','.join(segmenttype) if segmenttype and ('lane' in segmenttype or 'intersection' in segmenttype) else '')
    i += 1
plt.show()

In [ ]:
fov_lines = get_fov_lines(test_config)

In [ ]:
from sympy import Polygon, Line
test_segmentpolygon = list(map(Point, tuple(zip(*Geometry(segmentpolygon).exterior.shapely.xy))))
print(*test_segmentpolygon)
test_segmentpolygon = Polygon(*test_segmentpolygon)

In [ ]:
fov_lines

In [ ]:
test_segmentpolygon

In [ ]:
intersection(fov_lines, test_segmentpolygon)